<a href="https://colab.research.google.com/github/SylvainVerdy/chatbot/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Création d'un chatbot

# TODO LIST:



*   Apprendre les bases en python ([Apprendre les bases en python](https://www.youtube.com/playlist?list=PLlVRzxtr0mQmts12aa9wCs7jFuC8-8jLt))
*   Définir les étapes importantes pour réaliser ce chatbot
*   Télécharger les données pour créer un chatbot
*   Préparer et nettoyer les données, peut être en y ajoutant des stastiques (70-80% du temps du projet)
*   Définir le modèle pour créer le chatbot











In [1]:
import urllib3
import requests
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import spacy


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
!pip3 install transformers
!pip3 install elasticsearch

!mkdir "./chatbot/"

In [0]:
import transformers

In [4]:
url = "http://jmcauley.ucsd.edu/data/amazon/qa/icdm/QA_Video_Games.json.gz"
save_path = "./chatbot/QA_Video_Games.json.gz"
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    print(r)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)
download_url(url, save_path, chunk_size=128)
data_path = './chatbot/QA_video_games.txt'

import gzip
import shutil
with gzip.open(save_path, 'rb') as f_in:
    with open(data_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


<Response [200]>


In [0]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [73]:

# start server
import os
from subprocess import Popen, PIPE, STDOUT
es = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )
# wait a bit then test
!exec "./elasticsearch-7.0.0/bin/elasticsearch"

OpenJDK 64-Bit Server VM warning: Option UseConcMarkSweepGC was deprecated in version 9.0 and will likely be removed in a future release.
[2020-05-01T09:10:39,190][WARN ][o.e.b.ElasticsearchUncaughtExceptionHandler] [c482c481701b] uncaught exception in thread [main]
org.elasticsearch.bootstrap.StartupException: java.lang.RuntimeException: can not run elasticsearch as root
	at org.elasticsearch.bootstrap.Elasticsearch.init(Elasticsearch.java:163) ~[elasticsearch-7.0.0.jar:7.0.0]
	at org.elasticsearch.bootstrap.Elasticsearch.execute(Elasticsearch.java:150) ~[elasticsearch-7.0.0.jar:7.0.0]
	at org.elasticsearch.cli.EnvironmentAwareCommand.execute(EnvironmentAwareCommand.java:86) ~[elasticsearch-7.0.0.jar:7.0.0]
	at org.elasticsearch.cli.Command.mainWithoutErrorHandling(Command.java:124) ~[elasticsearch-cli-7.0.0.jar:7.0.0]
	at org.elasticsearch.cli.Command.main(Command.java:90) ~[elasticsearch-cli-7.0.0.jar:7.0.0]
	at org.elasticsearch.bootstrap.Elasticsearch.main(Elasticsearch.java:115) 

In [74]:
!curl -XGET "localhost:9200"

{
  "name" : "c482c481701b",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "8FCx4b6mSjmbusG6HThFuQ",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [0]:
import json, ast 
from pandas.io.json import json_normalize
from datetime import datetime
from elasticsearch import Elasticsearch

data_json = dict()
count = 0
data_list = []
es = Elasticsearch()

with open(data_path, "r") as file:
  data = file.readlines()
  for line in data:
    # data_json[count] = json.dumps(ast.literal_eval(line))
    data_list.append(json.dumps(ast.literal_eval(line)))
    res = es.index(index="test-index", id=count, body=json.dumps(ast.literal_eval(line)))
    res = es.get(index="test-index", id=count)
    es.indices.refresh(index="test-index")
    count +=1

res = es.search(index="test-index", body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total']['value'])


In [76]:
for hit in res['hits']['hits']:
    print(hit["_source"])

{'asin': 'B006IR6NH8', 'questions': [{'questionType': 'open-ended', 'askerID': 'A2QZUD6KQKW4ZP', 'questionTime': 'April 25, 2013', 'questionText': 'Is the item durable?', 'answers': [{'answerText': 'The wheel is very solid.  When you feel it you know it will last for a really long time.  I use mine all the time and have had no issues at all with anything breaking or even showing signs of wear.  It is a well built piece.', 'answererID': 'A3PYRCH7PCHDSB', 'answerTime': 'April 26, 2013', 'helpful': [2, 2]}, {'answerText': "The S1 wheel controller is nicely built, with a solid feel, hard rubber grips, cloth covered cable and large black allen bolts holding the shell together and phillips screws that affix the rubber handles.  The unit has a nice matt rubberized no glare finish. The plain round buttons seem durable as are the dials.  The gear shifter paddles seems reasonably strong, but would not handle major abuse or careless handling by young children.  It's not very heavy but has a bit o